In [ ]:
!pip install selenium pandas openpyxl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00


In [ ]:
import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException
import time

# Set up Chrome options to run in headless mode
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode (no GUI)
chrome_options.add_argument("--no-sandbox")  # Bypass OS security model
chrome_options.add_argument("--disable-dev-shm-usage")  # Overcome limited resource problems
chrome_options.add_argument("--disable-gpu")  # Disable GPU hardware acceleration

# Set up Selenium WebDriver with Chrome
def create_driver():
    return webdriver.Chrome(options=chrome_options)

# Function to scrape data for a specific country, year, and month
def scrape_data(driver, country, year, month, retries=3):
    try:
        print(f"Scraping data for {country}, {year}, {month}...")

        # Select country
        country_dropdown = Select(WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.NAME, 'cntcode'))
        ))
        country_dropdown.select_by_visible_text(country)

        # Set year type to Financial Year
        yeartype_radio = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.XPATH, '//input[@name="radioFY"]'))
        )
        yeartype_radio.click()

        # Select year
        year_dropdown = Select(WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.NAME, 'yy1'))
        ))
        year_dropdown.select_by_visible_text(str(year))

        # Select month
        month_dropdown = Select(WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.NAME, 'Mm1'))
        ))
        try:
            month_dropdown.select_by_visible_text(month.upper())  # Try uppercase first
        except:
            month_dropdown.select_by_visible_text(month)  # Try case-sensitive if uppercase fails

        # Set HS Code Level to 8-digit
        hs_code_dropdown = Select(WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.NAME, 'hslevel'))
        ))
        hs_code_dropdown.select_by_value('8')

        # Handle the "Sort On" dropdown
        sort_dropdown = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.NAME, 'sort'))
        )
        if sort_dropdown.tag_name == 'select':
            Select(sort_dropdown).select_by_visible_text('HS Code')

        # Set Display Records to TOP 100
        display_records_radio = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.XPATH, '//input[@name="radioD100"]'))
        )
        display_records_radio.click()

        # Select currency to USD
        currency_dropdown = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.NAME, 'radiousd'))
        )
        currency_dropdown.click()

        # Submit the form
        submit_button = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.NAME, 'button1'))
        )
        submit_button.click()

        # Wait for redirection and the table to appear
        WebDriverWait(driver, 40).until(
            EC.presence_of_element_located((By.TAG_NAME, 'table'))
        )

        # Scrape the table data
        table = driver.find_element(By.TAG_NAME, 'table')
        rows = table.find_elements(By.TAG_NAME, 'tr')
        data = []
        for row in rows[1:]:  # Skip the header row
            cols = row.find_elements(By.TAG_NAME, 'td')
            # Get text content from each column
            data.append([col.text.strip() for col in cols])

        # Define column headers based on your table structure
        columns = [
            'S.No.', 'HSCode', 'Commodity', 'Apr 2023 (R)', 'Apr 2024 (F)', '%Growth (1)',
            'Apr-Apr 2023 (R)', 'Apr-Apr 2024 (F)', '%Growth (2)'
        ]

        return pd.DataFrame(data, columns=columns)

    except (StaleElementReferenceException, TimeoutException) as e:
        print(f"Error during scraping for {country}, {year}, {month}: {e}")

        if retries > 0:
            print(f"Retrying... Attempts left: {retries}")
            time.sleep(5)  # Small delay before retrying
            return scrape_data(driver, country, year, month, retries=retries-1)

        print(f"Max retries reached for {country}, {year}, {month}. Skipping...")
        return None

# Create a folder to store CSV files
output_folder = 'scraped_data'
os.makedirs(output_folder, exist_ok=True)

# List of countries (A-Z)
countries = [
    "AFGHANISTAN", "ALBANIA", "ALGERIA", "ANDORRA", "ANGOLA", "ANTIGUA AND BARBUDA", "ARGENTINA", "ARMENIA", "AUSTRALIA", "AUSTRIA",
    "AZERBAIJAN", "BAHAMAS", "BAHRAIN", "BANGLADESH", "BARBADOS", "BELARUS", "BELGIUM", "BELIZE", "BENIN", "BHUTAN", "BOLIVIA",
    "BOSNIA AND HERZEGOVINA", "BOTSWANA", "BRAZIL", "BRUNEI DARUSSALAM", "BULGARIA", "BURKINA FASO", "BURUNDI", "CABO VERDE",
    "CAMBODIA", "CAMEROON", "CANADA", "CENTRAL AFRICAN REPUBLIC", "CHAD", "CHILE", "CHINA", "COLOMBIA", "COMOROS", "CONGO",
    "COSTA RICA", "CROATIA", "CUBA", "CYPRUS", "CZECH REPUBLIC", "DENMARK", "DJIBOUTI", "DOMINICA", "DOMINICAN REPUBLIC",
    "ECUADOR", "EGYPT", "EL SALVADOR", "EQUATORIAL GUINEA", "ERITREA", "ESTONIA", "ESWATINI", "ETHIOPIA", "FIJI", "FINLAND",
    "FRANCE", "GABON", "GAMBIA", "GEORGIA", "GERMANY", "GHANA", "GREECE", "GRENADA", "GUATEMALA", "GUINEA", "GUINEA-BISSAU",
    "GUYANA", "HAITI", "HONDURAS", "HUNGARY", "ICELAND", "INDIA", "INDONESIA", "IRAN", "IRAQ", "IRELAND", "ISRAEL", "ITALY",
    "JAMAICA", "JAPAN", "JORDAN", "KAZAKHSTAN", "KENYA", "KIRIBATI", "KOREA", "KUWAIT", "KYRGYZSTAN", "LAO PDR", "LATVIA",
    "LEBANON", "LESOTHO", "LIBERIA", "LIBYA", "LIECHTENSTEIN", "LITHUANIA", "LUXEMBOURG", "MADAGASCAR", "MALAWI", "MALAYSIA",
    "MALDIVES", "MALI", "MALTA", "MARSHALL ISLANDS", "MAURITANIA", "MAURITIUS", "MEXICO", "MICRONESIA", "MOLDOVA", "MONACO",
    "MONGOLIA", "MONTENEGRO", "MOROCCO", "MOZAMBIQUE", "MYANMAR", "NAMIBIA", "NAURU", "NEPAL", "NETHERLANDS", "NEW ZEALAND",
    "NICARAGUA", "NIGER", "NIGERIA", "NORTH MACEDONIA", "NORWAY", "OMAN", "PAKISTAN", "PALAU", "PANAMA", "PAPUA NEW GUINEA",
    "PARAGUAY", "PERU", "PHILIPPINES", "POLAND", "PORTUGAL", "QATAR", "ROMANIA", "RUSSIA", "RWANDA", "SAINT KITTS AND NEVIS",
    "SAINT LUCIA", "SAINT VINCENT AND THE GRENADINES", "SAMOA", "SAN MARINO", "SAO TOME AND PRINCIPE", "SAUDI ARABIA", "SENEGAL",
    "SERBIA", "SEYCHELLES", "SIERRA LEONE", "SINGAPORE", "SLOVAKIA", "SLOVENIA", "SOLOMON ISLANDS", "SOMALIA", "SOUTH AFRICA",
    "SOUTH SUDAN", "SPAIN", "SRI LANKA", "SUDAN", "SURINAME", "SWEDEN", "SWITZERLAND", "SYRIA", "TAJIKISTAN", "TANZANIA",
    "THAILAND", "TIMOR-LESTE", "TOGO", "TONGA", "TRINIDAD AND TOBAGO", "TUNISIA", "TURKEY", "TURKMENISTAN", "TUVALU", "UGANDA",
    "UKRAINE", "UNITED ARAB EMIRATES", "UNITED KINGDOM", "UNITED STATES", "URUGUAY", "UZBEKISTAN", "VANUATU", "VENEZUELA",
    "VIETNAM", "YEMEN", "ZAMBIA", "ZIMBABWE"
    # Add your list of countries here
]

# Years and months
years = range(2007, 2025)
months = ["JAN", "FEB", "MAR", "APR", "MAY", "JUN", "JUL", "AUG", "SEP", "OCT", "NOV", "DEC"]

# Create a driver instance
driver = create_driver()

try:
    for country in countries:
        for year in years:
            for month in months:
                try:
                    # Reload the page after each successful data scraping
                    driver.get('https://tradestat.commerce.gov.in/meidb/cntcomq.asp?ie=i')

                    # Add a small delay between requests to avoid overwhelming the server
                    time.sleep(3)

                    # Get the data
                    df = scrape_data(driver, country, year, month)

                    if df is not None and not df.empty:
                        filename = f"{country}_{month}_{year}.csv".replace(" ", "_")
                        file_path = os.path.join(output_folder, filename)

                        df.to_csv(file_path, index=False)
                        print(f"Data scraped and saved for {country}, {year}, {month} in {file_path}")

                except Exception as e:
                    print(f"Failed to scrape data for {country}, {year}, {month}. Error: {e}")

finally:
    # Close the browser after scraping is done
    driver.quit()


Scraping data for AFGHANISTAN, 2007, JAN...
Data scraped and saved for AFGHANISTAN, 2007, JAN in scraped_data/AFGHANISTAN_JAN_2007.csv
Scraping data for AFGHANISTAN, 2007, FEB...
Data scraped and saved for AFGHANISTAN, 2007, FEB in scraped_data/AFGHANISTAN_FEB_2007.csv
Scraping data for AFGHANISTAN, 2007, MAR...
Data scraped and saved for AFGHANISTAN, 2007, MAR in scraped_data/AFGHANISTAN_MAR_2007.csv
Scraping data for AFGHANISTAN, 2007, APR...
Data scraped and saved for AFGHANISTAN, 2007, APR in scraped_data/AFGHANISTAN_APR_2007.csv
Scraping data for AFGHANISTAN, 2007, MAY...
Error during scraping for AFGHANISTAN, 2007, MAY: Message: 
Stacktrace:
#0 0x59ab7af220aa <unknown>
#1 0x59ab7aa391a0 <unknown>
#2 0x59ab7aa85d06 <unknown>
#3 0x59ab7aa85fa1 <unknown>
#4 0x59ab7aacb344 <unknown>
#5 0x59ab7aaa9dad <unknown>
#6 0x59ab7aac8866 <unknown>
#7 0x59ab7aaa9b23 <unknown>
#8 0x59ab7aa78990 <unknown>
#9 0x59ab7aa7996e <unknown>
#10 0x59ab7aeee16b <unknown>
#11 0x59ab7aef1f68 <unknown>
#12 0x

KeyboardInterrupt: 